
---

# 🔐 4. **Dependency Injection System**

FastAPI’s **dependency injection system** lets you plug reusable logic (like DB access, config, auth) into endpoints cleanly — without repeating yourself. It's **Pythonic**, intuitive, and super powerful. 🧠

---

## 💡 4.1 `Depends()` – The Core

🔹 The `Depends()` function is used to **inject reusable code** (called dependencies) into your route handlers.

```python
from fastapi import Depends

def common_logic():
    return "Hello Dependency!"

@app.get("/inject")
def get_data(msg: str = Depends(common_logic)):
    return {"message": msg}
```

✅ Benefits:

* Reusability
* Separation of concerns
* Auto docs
* Clean & testable APIs

---

## 🔁 4.2 Shared Dependencies (DRY Pattern)

🔹 Avoid code duplication by declaring **shared dependencies** across multiple routes.

```python
def get_token_header(x_token: str = Header(...)):
    if x_token != "valid-token":
        raise HTTPException(status_code=403, detail="Unauthorized")
    return x_token

@app.get("/secure", dependencies=[Depends(get_token_header)])
def secure_data():
    return {"status": "Access granted"}
```

🧠 Tip: Use `dependencies=[...]` when you don’t need the return value — just enforce validation.

---

## 🪜 4.3 Sub-Dependencies (Layered Logic)

🔹 You can **nest dependencies** — one depends on another!

```python
def get_db():
    return "DB Connection"

def get_user(db=Depends(get_db)):
    return f"User from {db}"

@app.get("/me")
def read_user(user=Depends(get_user)):
    return {"user": user}
```

📌 This allows layering like:

* Auth depends on DB
* Logger depends on config
* Services depend on shared resources

---

## 🧱 4.4 Class-Based Dependencies

🔹 For complex use cases, define a class with `__call__()` method — it behaves like a dependency function.

```python
class AuthService:
    def __init__(self, token: str = Header(...)):
        self.token = token

    def __call__(self):
        if self.token != "admin-token":
            raise HTTPException(status_code=401, detail="Unauthorized")
        return self.token

@app.get("/admin")
def admin_data(token: str = Depends(AuthService())):
    return {"access": "admin granted"}
```

✅ Cleaner structure for services like:

* Auth
* DB connectors
* Caching layers
* Feature flags

---

## 🧠 Summary Table

| Concept                | What It Does                        | Real Use Case                    |
| ---------------------- | ----------------------------------- | -------------------------------- |
| `Depends()`            | Inject shared logic into endpoints  | DB, Auth, Config, Rate limits    |
| Shared Dependencies    | Apply same logic across routes      | Require token on multiple routes |
| Sub-dependencies       | Layer one dependency inside another | Auth → DB → Logger               |
| Class-based Dependency | Clean OOP injection pattern         | AuthService, CacheService, etc.  |

---

🧩 **Real-World Tip**:
Dependencies are also **automatically injected during testing**, making your API **more testable and scalable** 🧪✅

---
